# 【飞桨学习赛：中文场景文字识别】第七名方案
本项目为[常规赛：中文场景文字识别](https://aistudio.baidu.com/aistudio/competition/detail/20) 2022年8月份第七名的技术方案分享项目。

PaddleOCR参考: [https://github.com/PaddlePaddle/PaddleOCR ](https://github.com/PaddlePaddle/PaddleOCR )
##  项目简介：
本项目为基于PaddleOCR的中文场景文字识别，项目主要以CRNN网络为基础框架，结合数据增强及模型微调，采用ResNet34和MobileNetV3模型作为骨干网络，进行训练及预测。
![](https://ai-studio-static-online.cdn.bcebos.com/14e39136631c4c97ba8165f0de3c5c748c0158878d324ceabab1b16db5449fc9)

**注：本项目最后识别存在一定问题，最近比较忙，后续会处理，流程是对的！**

## 项目流程：
1.PaddleOCR安装

2.数据处理

3.模型调整

4.训练与预测



# 一、环境安装
## 1.1安装PaddleOCR



In [ ]:
#克隆PaddleOCR项目
!cd ~/work && git clone -b develop https://gitee.com/paddlepaddle/PaddleOCR.git

In [ ]:
#安装提供环境文件requirements.txt
!cd ~/work/PaddleOCR 
！pip install -r requirements.txt && python setup.py install

# 二、数据处理
## 2.1解压数据集

In [ ]:
#解压训练集与测试集
!cd ~/data/data62842/ && unzip train_images.zip
!cd ~/data/data62843/ && unzip test_images.zip

In [ ]:
!cd ~/data/data62842/ && mv train_images ../ && mv train_label.csv ../
!cd ~/data/data62843/ && mv test_images ../ 

In [5]:
%cd data/data62842
!cat mv train_label.csv | head  -n 10

/home/aistudio/data/data62842
cat: mv: No such file or directory
name,value
0.jpg,����
1.jpg,����
2.jpg,����
3.jpg,ǰ�Ŵ�դ���ܵ�
4.jpg,������������
5.jpg,ë�³���ֱ��
6.jpg,13761916218
7.jpg,�����ײ�
8.jpg,��������
cat: write error: Broken pipe


## 2.2数据增广
* 首先，考虑使用轻量模型会有一定精度损失，采用经典网络ResNet34。
* 其次，为了进一步增强识别效果及模型泛化行，参考其他项目使用[text_render](https://github.com/Sanster/text_renderer)进行数据增广。
* 最后，使用text_renderer进行数据增广，修改text_render/configs/default.yaml配置，以下为更改后的模版，主要将三项做修改，分别是font_color的enable设为True，img_bg的enable设为False，seamless_clone的enable设为True。
    

数据增广参考本项目：https://aistudio.baidu.com/aistudio/projectdetail/1306342?channelType=0&channel=0

```
# Small font_size will make text looks like blured/prydown
font_size:
  min: 14
  max: 23

# choose Text color range
# color boundary is in R,G,B format
font_color:
  enable: True
  blue:
    fraction: 0.5
    l_boundary: [0,0,150]
    h_boundary: [60,60,255]
  brown:
    fraction: 0.5
    l_boundary: [139,70,19]
    h_boundary: [160,82,43]

# By default, text is drawed by Pillow with (https://stackoverflow.com/questions/43828955/measuring-width-of-text-python-pil)
# If `random_space` is enabled, some text will be drawed char by char with a random space
random_space:
  enable: false
  fraction: 0.3
  min: -0.1 # -0.1 will make chars very close or even overlapped
  max: 0.1

# Do remap with sin()
# Currently this process is very slow!
curve:
  enable: false
  fraction: 0.3
  period: 360  # degree, sin 函数的周期
  min: 1 # sin 函数的幅值范围
  max: 5

# random crop text height
crop:
  enable: false
  fraction: 0.5

  # top and bottom will applied equally
  top:
    min: 5
    max: 10 # in pixel, this value should small than img_height
  bottom:
    min: 5
    max: 10 # in pixel, this value should small than img_height

# Use image in bg_dir as background for text
img_bg:
  enable: false
  fraction: 0.5

# Not work when random_space applied
text_border:
  enable: false
  fraction: 0.5

  # lighter than word color
  light:
    enable: true
    fraction: 0.5

  # darker than word color
  dark:
    enable: true
    fraction: 0.5

# https://docs.opencv.org/3.4/df/da0/group__photo__clone.html#ga2bf426e4c93a6b1f21705513dfeca49d
# https://www.cs.virginia.edu/~connelly/class/2014/comp_photo/proj2/poisson.pdf
# Use opencv seamlessClone() to draw text on background
# For some background image, this will make text image looks more real
seamless_clone:
  enable: true
  fraction: 0.5

perspective_transform:
  max_x: 25
  max_y: 25
  max_z: 3

blur:
  enable: true
  fraction: 0.03

# If an image is applied blur, it will not be applied prydown
prydown:
  enable: true
  fraction: 0.03
  max_scale: 1.5 # Image will first resize to 1.5x, and than resize to 1x

noise:
  enable: true
  fraction: 0.3

  gauss:
    enable: true
    fraction: 0.25

  uniform:
    enable: true
    fraction: 0.25

  salt_pepper:
    enable: true
    fraction: 0.25

  poisson:
    enable: true
    fraction: 0.25

line:
  enable: false
  fraction: 0.05

  random_over:
    enable: true
    fraction: 0.2

  under_line:
    enable: false
    fraction: 0.2

  table_line:
    enable: false
    fraction: 0.3

  middle_line:
    enable: false
    fraction: 0.3

line_color:
  enable: false
  black:
    fraction: 0.5
    l_boundary: [0,0,0]
    h_boundary: [64,64,64]
  blue:
    fraction: 0.5
    l_boundary: [0,0,150]
    h_boundary: [60,60,255]

# These operates are applied on the final output image,
# so actually it can also be applied in training process as an data augmentation method.

# By default, text is darker than background.
# If `reverse_color` is enabled, some images will have dark background and light text
reverse_color:
  enable: false
  fraction: 0.5

emboss:
  enable: false
  fraction: 0.1

sharp:
  enable: false
  fraction: 0.1


```

In [ ]:
!cd ~/work && git clone https://github.com/Sanster/text_renderer
!cd ~/work/text_renderer && pip install -r requirements.txt

* CRNN网络主要考虑图片高度。以4为倍数，通过统计训练集的图像尺寸，训练时图片高度设为48。

In [ ]:
import glob
import os
import cv2


def get_aspect_ratio(img_set_dir):
    m_width = 0
    m_height = 0
    width_dict = {}
    height_dict = {}
    images = glob.glob(img_set_dir+'*.jpg')
    for image in images:
        img = cv2.imread(image)
        width_dict[int(img.shape[1])] = 1 if (int(img.shape[1])) not in width_dict else 1 + width_dict[int(img.shape[1])]
        height_dict[int(img.shape[0])] = 1 if (int(img.shape[0])) not in height_dict else 1 + height_dict[int(img.shape[0])]
        m_width += img.shape[1]
        m_height += img.shape[0]
    m_width = m_width/len(images)
    m_height = m_height/len(images)
    aspect_ratio = m_width/m_height
    width_dict = dict(sorted(width_dict.items(), key=lambda item: item[1], reverse=True))
    height_dict = dict(sorted(height_dict.items(), key=lambda item: item[1], reverse=True))
    return aspect_ratio,m_width,m_height,width_dict,height_dict
aspect_ratio,m_width,m_height,width_dict,height_dict = get_aspect_ratio("/home/aistudio/data/train_images/")
print("aspect ratio is: {}, mean width is: {}, mean height is: {}".format(aspect_ratio,m_width,m_height))
print("Width dict:{}".format(width_dict))
print("Height dict:{}".format(height_dict))
import pandas as pd

def Q2B(s):
    """全角转半角"""
    inside_code=ord(s)
    if inside_code==0x3000:
        inside_code=0x0020
    else:
        inside_code-=0xfee0
    if inside_code<0x0020 or inside_code>0x7e: #转完之后不是半角字符返回原来的字符
        return s
    return chr(inside_code)

def stringQ2B(s):
    """把字符串全角转半角"""
    return "".join([Q2B(c) for c in s])

def is_chinese(s):
    """判断unicode是否是汉字"""
    for c in s:
        if c < u'\u4e00' or c > u'\u9fa5':
            return False
    return True

def is_number(s):
    """判断unicode是否是数字"""
    for c in s:
        if c < u'\u0030' or c > u'\u0039':
            return False
    return True

def is_alphabet(s):
    """判断unicode是否是英文字母"""
    for c in s:
        if c < u'\u0061' or c > u'\u007a':
            return False
    return True

def del_other(s):
    """判断是否非汉字，数字和小写英文"""
    res = str()
    for c in s:
        if not (is_chinese(c) or is_number(c) or is_alphabet(c)):
            c = ""
        res += c
    return res


df = pd.read_csv("/home/aistudio/data/train_label.csv", encoding="gbk")
name, value = list(df.name), list(df.value)
for i, label in enumerate(value):
    # 全角转半角
    label = stringQ2B(label)
    # 大写转小写
    label = "".join([c.lower() for c in label])
    # 删除所有空格符号
    label = del_other(label)
    value[i] = label

# 删除标签为""的行
data = zip(name, value)
data = list(filter(lambda c: c[1]!="", list(data)))
# 保存到work目录
with open("/home/aistudio/data/train_label.txt", "w") as f:
    for line in data:
        f.write(line[0] + "\t" + line[1] + "\n")

# 记录训练集中最长标签
label_max_len = 0
with open("/home/aistudio/data/train_label.txt", "r") as f:
    for line in f:
        name, label = line.strip().split("\t")
        if len(label) > label_max_len:
            label_max_len = len(label)

print("label max len: ", label_max_len)
def create_label_list(train_list):
    classSet = set()
    with open(train_list) as f:
        next(f)
        for line in f:
            img_name, label = line.strip().split("\t")
            for e in label:
                classSet.add(e)
    # 在类的基础上加一个blank
    classList = sorted(list(classSet))
    with open("/home/aistudio/data/label_list.txt", "w") as f:
        for idx, c in enumerate(classList):
            f.write("{}\t{}\n".format(c, idx))
    
    # 为数据增广提供词库
    with open("/home/aistudio/work/text_renderer/data/chars/ch.txt", "w") as f:
        for idx, c in enumerate(classList):
            f.write("{}\n".format(c))
            
    return classSet

classSet = create_label_list("/home/aistudio/data/train_label.txt")
print("classify num: ", len(classSet))

aspect ratio is: 3.451128333333333, mean width is: 165.65416, mean height is: 48.0
Width dict:{48: 741, 96: 539, 44: 392, 42: 381, 144: 365, 45: 345, 43: 323, 72: 318, 88: 318, 40: 312, 52: 301, 36: 298, 50: 297, 120: 294, 54: 288, 84: 286, 51: 283, 32: 283, 24: 281, 100: 277, 64: 276, 80: 276, 76: 275, 102: 272, 81: 270, 90: 269, 56: 268, 66: 267, 78: 266, 37: 262, 82: 261, 41: 259, 89: 258, 92: 257, 46: 256, 60: 251, 86: 249, 168: 246, 53: 246, 105: 243, 61: 242, 57: 241, 128: 241, 112: 240, 85: 239, 39: 237, 91: 237, 68: 235, 98: 234, 192: 233, 93: 233, 75: 232, 74: 229, 34: 229, 33: 229, 70: 228, 25: 226, 87: 226, 104: 224, 110: 224, 58: 223, 132: 221, 62: 221, 49: 221, 101: 221, 108: 221, 126: 218, 150: 218, 94: 217, 73: 217, 129: 216, 28: 216, 30: 215, 69: 215, 99: 212, 160: 211, 38: 210, 136: 209, 109: 207, 26: 207, 55: 206, 35: 205, 118: 205, 116: 204, 115: 203, 174: 201, 117: 200, 148: 200, 106: 200, 122: 199, 113: 198, 67: 197, 77: 197, 172: 195, 114: 195, 156: 194, 130: 191,

- 生成字符长度为1，2，3，4，5的数据集各2000张，共10000张。

- 部分生成图片示例如：

![](https://ai-studio-static-online.cdn.bcebos.com/70da7b3787df4153a3de95e6dd80c6f045c663e7f67443bd92d6cb3e3bb40aab)
![](https://ai-studio-static-online.cdn.bcebos.com/91b8dcf3ea504583b8633a3cb443cd71cf05734918d54d6fae1cf17a54cc6fb0)
![](https://ai-studio-static-online.cdn.bcebos.com/7dff3df80452482ab1ac858ad3ac567f899074ff3f5b40f0b966a5452915e431)
![](https://ai-studio-static-online.cdn.bcebos.com/aed00f66ec284f068e5b4f4e834df2b8258cbd04f6294fb395fca3d1068a2de6)
![](https://ai-studio-static-online.cdn.bcebos.com/d041f7be6cac4031bc8c94a7ec41674ed6a3857b480446af9e3d45c3c6dcd44e)




In [ ]:
!cd ~/work/text_renderer && python main.py --length 1 --img_width 32 --img_height 48 --chars_file "./data/chars/ch.txt" --corpus_mode 'random' --num_img 2000
!cd ~/work/text_renderer && python main.py --length 2 --img_width 64 --img_height 48 --chars_file "./data/chars/ch.txt" --corpus_mode 'random' --num_img 2000
!cd ~/work/text_renderer && python main.py --length 3 --img_width 96 --img_height 48 --chars_file "./data/chars/ch.txt" --corpus_mode 'random' --num_img 2000
!cd ~/work/text_renderer && python main.py --length 4 --img_width 128 --img_height 48 --chars_file "./data/chars/ch.txt" --corpus_mode 'random' --num_img 2000
!cd ~/work/text_renderer && python main.py --length 5 --img_width 160 --img_height 48 --chars_file "./data/chars/ch.txt" --corpus_mode 'random' --num_img 2000

Total fonts num: 1
Background num: 1
Generate text images in ./output/default
2000/2000 100%
Finish generate data: 5.758 s
Total fonts num: 1
Background num: 1
Generate more text images in ./output/default. Start index 2000
2000/2000 100%
Finish generate data: 8.900 s
Total fonts num: 1
Background num: 1
Generate more text images in ./output/default. Start index 4000
2000/2000 100%
Finish generate data: 11.872 s
Total fonts num: 1
Background num: 1
Generate more text images in ./output/default. Start index 6000
2000/2000 100%
Finish generate data: 16.067 s
Total fonts num: 1
Background num: 1
Generate more text images in ./output/default. Start index 8000
2000/2000 100%
Finish generate data: 19.593 s


- 将生成的数据集与原数据集合并

In [ ]:
!cp ~/work/text_renderer/output/default/*.jpg ~/data/train_images

In [ ]:
import os

with open('work/text_renderer/output/default/tmp_labels.txt','r',encoding='utf-8') as src_label:
    with open('data/train_label.txt','a',encoding='utf-8') as dst_label:
        lines = src_label.readlines()
        for line in lines:
            [img,text] = line.split(' ')
            print('{}.jpg\t{}'.format(img,text),file=dst_label,end='')

# 三、模型调整
* 加载CRNN预训练模型
* 改变默认输入图片尺寸，变为为height48，width256
* 优化学习率策略，通过cosine_decay和warmup策略加快模型收敛

## CRNN模型介绍

* 本项目模型采用文字识别经典CRNN模型(CNN+RNN+CTC),其中部分模型代码经过PaddleOCR源码改编，完成识别模型的搭建、训练、评估和预测过程。训练时可以手动更改config配置文件（数据训练、加载、评估验证等参数），默认采用优化器采用Adam,使用CTC损失函数。本项目采用ResNet34作为骨干网络。

![](https://ai-studio-static-online.cdn.bcebos.com/af456f671aeb41c7be60d28379d93dd3449bf4f261614d9ea09452ccd38cb7e0)
```
CRNN网络结构包含三部分，从下到上依次为：

（1）卷积层。作用是从输入图像中提取特征序列。

（2）循环层。作用是预测从卷积层获取的特征序列的标签（真实值）分布。

（3）转录层。作用是把从循环层获取的标签分布通过去重整合等操作转换成最终的识别结果。
```






In [ ]:
!cd ~/work/PaddleOCR && mkdir pretrain_weights && cd pretrain_weights && wget https://paddleocr.bj.bcebos.com/20-09-22/server/rec/ch_ppocr_server_v1.1_rec_pre.tar

--2022-09-30 17:16:20--  https://paddleocr.bj.bcebos.com/20-09-22/server/rec/ch_ppocr_server_v1.1_rec_pre.tar
Resolving paddleocr.bj.bcebos.com (paddleocr.bj.bcebos.com)... 220.181.33.43, 220.181.33.44, 2409:8c04:1001:1002:0:ff:b001:368a
Connecting to paddleocr.bj.bcebos.com (paddleocr.bj.bcebos.com)|220.181.33.43|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 297577776 (284M) [application/x-tar]
Saving to: ‘ch_ppocr_server_v1.1_rec_pre.tar’

ch_ppocr_server_v1. 100%[===================>] 283.79M  46.8MB/s    in 5.3s    

2022-09-30 17:16:26 (53.2 MB/s) - ‘ch_ppocr_server_v1.1_rec_pre.tar’ saved [297577776/297577776]



In [ ]:
!cd ~/work/PaddleOCR/pretrain_weights && tar -xf ch_ppocr_server_v1.1_rec_pre.tar

* 在PaddleOCR/configs/rec中，添加训练配置文件 my_rec_ch_train.yml和my_rec_ch_reader.yml

```
#my_rec_ch_train.yml
Global:
  algorithm: CRNN
  use_gpu: true
  epoch_num: 201
  log_smooth_window: 20
  print_batch_step: 10
  save_model_dir: ./output/my_rec_ch
  save_epoch_step: 50
  eval_batch_step: 100000000
  train_batch_size_per_card: 64
  test_batch_size_per_card: 64
  image_shape: [3, 48, 256]
  max_text_length: 80
  character_type: ch
  character_dict_path: ./ppocr/utils/ppocr_keys_v1.txt
  loss_type: ctc
  distort: true
  use_space_char: true
  reader_yml: ./configs/rec/my_rec_ch_reader.yml
  pretrain_weights: ./pretrain_weights/ch_ppocr_server_v1.1_rec_pre/best_accuracy
  checkpoints:
  save_inference_dir: 
  infer_img:

Architecture:
  function: ppocr.modeling.architectures.rec_model,RecModel

Backbone:
  function: ppocr.modeling.backbones.rec_resnet_vd,ResNet
  layers: 34

Head:
  function: ppocr.modeling.heads.rec_ctc_head,CTCPredict
  encoder_type: rnn
  fc_decay: 0.00004
  SeqRNN:
    hidden_size: 256
    
Loss:
  function: ppocr.modeling.losses.rec_ctc_loss,CTCLoss

Optimizer:
  function: ppocr.optimizer,AdamDecay
  base_lr: 0.0001
  l2_decay: 0.00004
  beta1: 0.9
  beta2: 0.999
  decay:
    function: cosine_decay_warmup
    step_each_epoch: 1000
    total_epoch: 201
    warmup_minibatch: 2000
```

```
#my_rec_ch_reader.yml
TrainReader:
  reader_function: ppocr.data.rec.dataset_traversal,SimpleReader
  num_workers: 1
  img_set_dir: /home/aistudio/data/train_images
  label_file_path: /home/aistudio/data/train_label.txt
  
EvalReader:
  reader_function: ppocr.data.rec.dataset_traversal,SimpleReader
  img_set_dir: /home/aistudio/data/train_images
  label_file_path: /home/aistudio/data/train_label.txt

TestReader:
  reader_function: ppocr.data.rec.dataset_traversal,SimpleReader
```

# 四、训练与预测



## 4.1 训练模型
- 根据修改后的配置文件，输入以下命令就可以开始训练。

In [33]:
!pwd
!cd ~/work/PaddleOCR && python tools/train.py -c configs/rec/my_rec_ch_train.yml

## 4.2导出模型
通过export_model.py导出模型，设置配置文件及导出路径。

In [34]:
!cd ~/work/PaddleOCR && python tools/export_model.py -c configs/rec/my_rec_ch_train.yml -o Global.checkpoints=./output/my_rec_ch/iter_epoch_27 Global.save_inference_dir=./inference/CRNN_R34

2022-09-30 22:57:53,971-INFO: {'Global': {'debug': False, 'algorithm': 'CRNN', 'use_gpu': True, 'epoch_num': 201, 'log_smooth_window': 20, 'print_batch_step': 10, 'save_model_dir': './output/my_rec_ch', 'save_epoch_step': 3, 'eval_batch_step': 100000000, 'train_batch_size_per_card': 64, 'test_batch_size_per_card': 64, 'image_shape': [3, 48, 256], 'max_text_length': 80, 'character_type': 'ch', 'character_dict_path': './ppocr/utils/ppocr_keys_v1.txt', 'loss_type': 'ctc', 'distort': True, 'use_space_char': True, 'reader_yml': './configs/rec/my_rec_ch_reader.yml', 'pretrain_weights': './pretrain_weights/ch_ppocr_server_v1.1_rec_pre/best_accuracy', 'checkpoints': './output/my_rec_ch/iter_epoch_27', 'save_inference_dir': './inference/CRNN_R34', 'infer_img': None}, 'Architecture': {'function': 'ppocr.modeling.architectures.rec_model,RecModel'}, 'Backbone': {'function': 'ppocr.modeling.backbones.rec_resnet_vd,ResNet', 'layers': 34}, 'Head': {'function': 'ppocr.modeling.heads.rec_ctc_head,CTCPr

## 4.3预测结果

修改模型路径，运行predict.py:
```
import sys 
import os

from paddleocr import PaddleOCR
import numpy as np
import glob
import time
if __name__=='__main__':

    # Preference
    img_set_dir = os.path.join('..','data','test_images','')
    
    # Load model
    use_gpu = True 
    use_angle_cls = False 
    det = False
    det_model_dir = os.path.join('PaddleOCR','inference','ch_ppocr_mobile_v1.1_det_infer')
    cls_model_dir = os.path.join('PaddleOCR','inference','ch_ppocr_mobile_v1.1_cls_infer')
    rec_model_dir = os.path.join('PaddleOCR','inference','CRNN_R34')
    ocr = PaddleOCR(use_angle_cls=use_angle_cls, lang="ch",use_gpu=use_gpu,use_space_char=False,gpu_mem=4000,
                    det = det,
                    rec_image_shape = '3, 48, 256', 
                    rec_algorithm = 'CRNN',
                    max_text_length = 80,
                    det_model_dir = det_model_dir,
                    cls_model_dir = cls_model_dir,
                    rec_model_dir = rec_model_dir
                    )

    # Load data in a folder
    images = glob.glob(img_set_dir+'*.jpg')

    log_file_name = time.strftime('%Y-%m-%d-%H-%M-%S', time.localtime())
    # Print result to a file
    with open(log_file_name+'.txt','w') as fid:
        print('new_name\tvalue',file=fid)
        #Inference in a folder
        for image in images:
            result = ocr.ocr(image, cls=use_angle_cls,det=det)
            if result is None:
                print('Test {} failed.'.format(image.replace(img_set_dir,'')))
                continue

            for info in result:
                pred_label = info[0]
                print('{}\t{}'.format(image.replace(img_set_dir,''),pred_label),file=fid)
    print("Finished predicting {} images!".format(len(images)))
```


In [35]:
!pwd
!python ~/work/predict.py

/home/aistudio


In [17]:
#查看结果 txt文件生成
%cd /home/aistudio/
!cat 2022-09-30-22-58-06.txt | head -n 2

/home/aistudio
new_name	value
2883.jpg	epo
cat: write error: Broken pipe


# 参考资料
- [PaddleOCR官方教程](https://github.com/PaddlePaddle/PaddleOCR/blob/develop/doc/doc_ch/recognition.md)  
- [PaddleOCR：中文场景文字识别](https://aistudio.baidu.com/aistudio/projectdetail/588846)
